# TP2 : Perceptron multicouches pour l'analyse de sentiment

Le but de ce TP va être d'entraîner et de comparer plusieurs réseaux de neurones de type et de taille différent sur une tâche textuelle, à savoir l'analyse de sentiment.
Maintenant que nous savons comment fonctionne un réseau de neurones, nous allons pouvoir utiliser des bibliothèques Python qui vont grandement facilitent grandement la création et l'entraînement de réseaux.

Ce TP ne demande pas d'écriture de code, n'hésitez donc pas à modifier les divers paramètres pour observer ce qui se passe !

**Date limite de rendu :** Le 11 octobre 2024 à 10:30.

## Préparatifs

Les commandes suivantes permettent d'installer et de charger les bibliothèques Python nécessaires.

In [ ]:
# Installation de keras et de tensorflow (tensorflow étant une grosse bibliothèque, cela peut prendre un certain temps)
# Cette cellule n'a besoin d'être exécutée qu'une seule fois en tout et pour tout
!pip install keras Keras-Preprocessing tensorflow nltk

**Attention :** Après avoir exécuté la cellule ci-dessus, il faut impérativement redémarrer le noyau Python pour que Python puisse reconnaître le module nouvellement installé.

La manière de faire cela dépend de votre éditeur :

- Sur Jupyter : `Noyau -> Redémarrer` (ou `Kernel -> Restart` si l'instance est en anglais)
- Sur Colab : `Runtime -> Restart session` : <div><img src="img/colab.png" width="500"/></div>
- Sur VSCode : Un bouton `Restart`/`Redémarrer` : <div><img src="img/vscode.png" width="500"/></div>



In [ ]:
# Importation des bibliothèques:
# - random pour la génération de nombres aléatoires et la reproducibilité
# - numpy pour la gestion des matrices
# - tensorflow pour la gestion de réseaux de neuroens
# - keras pour la création et l'entraînement simplifié des réseaux (keras est une interface haut niveau qui utilise tensorflow)
# - pandas pour le chargement et la visualisation des données sous forme de tableau
# - scikit-learn, nltk et re pour le pré-traitement des données
# - matplotlib pour les graphiques

import random
import re
import numpy as np
import tensorflow as tf
import pandas as pd
import nltk
import keras

import matplotlib.pyplot as plt

from keras import Sequential
from keras.layers import Dense
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, classification_report

# Reproducibilité
seed = 1

# Préparation des données

Le jeu de données sur lequel nous allons travailler est issu de [Kaggle](https://www.kaggle.com/datasets/abhi8923shriv/sentiment-analysis-dataset). Il s'agit d'un ensemble de tweets qui ont été automatiquement annoté pour de l'analyse de sentiment. Autrement dit, pour un tweet donné, on cherche à savoir si son contenu est positif, négatif ou neutre.

In [ ]:
# Chargement des données
df = pd.read_csv('data.csv',encoding='latin1')
df

Le jeu de données contient, en plus du tweet et de son étiquette (positif/négatif/neutre), beaucoup d'informations supplémentaires comme l'âge et le pays de l'utilisateur, ainsi que l'heure de la journée à laquelle le tweet a été écrit. Dans ce TP, nous n'utiliserons que le texte.

Notre but va être de prédire l'étiquette d'un tweet à partir de son texte. Il s'agit donc à nouveau d'un problème de **classification**.

Comme la dernière fois, la première étape consiste à transformer les étiquettes en probabilités. Nous avons à nouveau 3 types d'étiquettes possible, donc 3 probabilités pour chaque point de données. 

In [ ]:
# On garde uniquement les deux colonnes qui nous intéressent, et on se débarrasse des lignes contenant des champs vides (N/A)
df = df[['text', 'sentiment']].dropna()

enc = OneHotEncoder(sparse_output=False)
labels = enc.fit_transform(df['sentiment'].values.reshape(-1, 1))
labels

Il nous faut désormais transformer le texte de chaque tweet en données numériques. Beaucoup d'approches sont possibles pour cela, dont en voici certaines :
- L'encodage 1 parmi n (vu précédemment), qui transforme chaque mot en une liste de zéros contenant un seul 1 à une position correspondant au mot
- La TF-IDF, qui transforme un texte en une liste de fréquences relatives de mots
- Les plongements sémantiques (*embedding*), qui transforme chaque mot du texte initial en une liste de nombres calibrée de manière à représenter le sens du mot

Nous allons dans un premier temps nous focaliser sur la TF-IDF.

# TF-IDF

La [TF-IDF](https://fr.wikipedia.org/wiki/TF-IDF) (term frequency/inverse document frequency) est une métrique qui associe à un document (texte) et à un mot donné un score d'importance compris entre 0 et 1. Nous ne nous attarderons pas sur sa formulation mathématique au cours de ce TP, mais elle utilise les principes suivants :
- Si un mot est fréquent dans un document mais dans peu d'autres documents, alors ce mot est spécifique à ce document et a un score élevé
- Si un mot est fréquent dans un grand nombre de documents, alors ce mot est commun et son score est plus faible.

La librairie `scikit-learn` contient un outil permettant de générer automatiquement la TF-IDF d'un ensemble de documents; elle s'occupe de diviser chaque document en mot, puis calcule le score de chaque mot.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()

# Calcule la TF-IDF de chaque mot dans chaque texte
tfidf = vectorization.fit_transform(df['text'])

La matrice `tfidf` contient désormais en position (i, j) la TF-IDF du mot numéro `j` dans le document numéro `i`.
Les numéros associés à chaque mot peuvent être lus en accèdant au dictionnaire `vectorization.vocabulary_`.

**À noter :** Par défault, `TfidfVectorizer` transforme tout le texte en minuscule et ignore une partie de la ponctuation.

Par exemple :

In [ ]:
num_doc = 1
print('Document :', df['text'][num_doc])

num_mot = vectorization.vocabulary_['in']
print('TF-IDF pour "in" :   ', tfidf[num_doc, num_mot])

num_mot = vectorization.vocabulary_['diego']
print('TF-IDF pour "diego" :', tfidf[num_doc, num_mot])

num_mot = vectorization.vocabulary_['dog']
print('TF-IDF pour "dog" :  ', tfidf[num_doc, num_mot])


### Séparation et mélange

On rappelle qu'en apprentissage machine, il est important de séparer ses données en trois jeux :
- Le jeu d'entraînement, servant à entraîner les modèles,
- Le jeu de validation, qui sert à mesurer les performances des divers modèles que nous allons entraîner et à les comparer
- Le jeu de test, qui sert à mesurer les performances finales (vraies) du modèle retenu.

Nous pouvons réaliser la séparation grâce à la fonction `train_test_split`, comme au TP 1.

En général, on choisit de garder la majorité des données pour l'entraînement, et autant de données pour la validation que pour le test. Les ratios les plus couramment utilisés pour les trois jeux sont 60%-20%-20%, 80%-10%-10% et 90%-5%-5%.

In [ ]:
train_frac = 0.8
valid_frac = 0.1
test_frac = 0.1

# On sépare d'abord le jeu d'entraînement du reste
X_train, X_2, y_train, y_2 = train_test_split(tfidf, labels, test_size=valid_frac + test_frac, shuffle=True, random_state=seed)
# Puis on sépare le reste en deux (validation et test)
X_valid, X_test, y_valid, y_test = train_test_split(X_2, y_2, test_size=test_frac/(test_frac + valid_frac), shuffle=True, random_state=seed)

print(X_train.shape)
print(y_train.shape)

print(X_valid.shape)
print(y_valid.shape)

print(X_test.shape)
print(y_test.shape)

**Q1 :** L'ensemble d'entraînement contient 21 984 documents, et nous avons 3 étiquettes par exemple. À quoi correspond le nombre 26 397 ?

**Réponse :** 26 397 correspond à la largeur de la matrice TF-IDF, autrement dit le nombre de mots uniques trouvés dans l'intégralité du corpus. Chaque document obtient 26 397 scores (un pour chaque mot). Si un mot n'est pas présent dans un document, le score de ce mot pour ce document est automatiquement 0. La matrice contient donc en majorité des 0.

# Réseau de neurones

Comme au premier TP, nous allons commencer par créer un réseau de neurones à une couche (entrée -> sortie).

En plus de cela, notre réseau de neurones utilisera les éléments suivants :
- Fonction d'activation : Softmax (utilisée dans le cas d'une classification à plus de deux classes, alors que la sigmoïde est plutôt utilisée dans le cas à deux classes)
- Fonction de coût : Entropie croisée (utilisée en classification)

Enfin, pour effectuer la descente de gradient, diverses méthodes existent. La méthode que nous avons utilisée la dernière fois est la plus simple, mais elle n'est pas très efficace. Beaucoup d'autres méthodes plus complexes existent, faisant par exemple varier le taux d'apprentissage au cours du temps (*momentum*) ou en gardant en mémoire un taux d'apprentissage différent pour chaque poids du réseau, par exemple. La composante qui s'occupe de gérer cette partie s'appelle un *optimiseur*. Nous utiliserons ici l'optimiseur [Adam](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Adam).

Par chance, Keras rend l'implémentation de tout ceci extrêmement simple :

In [ ]:
# Reproducibilité
keras.utils.set_random_seed(seed)

# Création d'un réseau de neurones (liste de couches)
model = Sequential()

# Ajout d'une couche simple (dense), avec 3 sorties, et utilisant le Softmax comme fonction d'application
model.add(Dense(3, activation='softmax'))

# Finalise la création du modèle en utilisant l'entropie croisée pour fonction de coût, l'optimiseur Adam et en mesurant la précision du modèle à chaque époque
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entraîne le modèle pendant 30 itérations sur le jeu d'entraînement, et en mesurant la précision sur les jeux d'entraînement et de validation
# Cet étape prend une minute environ
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

# Affiche les couches du modèles et leur nombre de poids (paramètres)
model.summary()

Pratique, non ?

On peut également facilement visualiser l'évolution de la précision au cours de l'entraînement :

In [ ]:
def display_accuracy(history):
    plt.plot(history.history['accuracy'], label='Précision (entraînement)')
    plt.plot(history.history['val_accuracy'], label='Précision (validation)')
    plt.xlabel('Itération')
    plt.legend()
    plt.grid()
    plt.show()

display_accuracy(history)

**Q2 :** D'après ce graphique, à quelle itération arrêteriez-vous l'entraînement et pourquoi ?

**Réponse :** Il est ici possible de s'arrêter à la 15e itération. En effet, la précision sur l'ensemble de validation (qui témoigne des performances réelles du modèle) n'augmente plus. La précision d'entraînement continue d'augmenter, mais cela témoigne uniquement d'un surapprentissage possible.

# Utilisation du réseau

Faisons un test sur un exemple simple :

In [ ]:
ex_text = "The movie was so bad, I will not recommend this movie to anyone"

ex_tfidf = vectorization.transform([ex_text])
out = model(ex_tfidf.toarray())
out

Comment savoir quelle probabilité correspond à telle étiquette ? Nous pouvons regarder le contenu de l'encodeur 1 parmi n pour répondre à cette question :

In [ ]:
print(enc.inverse_transform([[1, 0, 0]]))
print(enc.inverse_transform([[0, 1, 0]]))
print(enc.inverse_transform([[0, 0, 1]]))

La première probabilité correspond donc à l'étiquette négative, la deuxième à la neutre et la dernière à la positive. Créons une fonction pour automatiser tout cela :

In [ ]:
def get_label(pos):
    label = [0, 0, 0]
    label[pos] = 1
    return enc.inverse_transform([label])[0][0]

def predict(ex_text, model, display=True):
    ex_tfidf = vectorization.transform([ex_text])
    out = model(ex_tfidf.toarray())
    out = out.numpy()
    sentiment = get_label(out.argmax())
    if display:
        return ex_text + ' -> ' + sentiment
    return sentiment

print(predict("I feel sad today", model))
print(predict("I love my mom", model))
print(predict("Today is Tuesday", model))
print(predict("Good riddance!", model))  # <- Erreur de classification

Comme ce réseau ne contient qu'une seule couche, on peut considérer qu'il attribue en fait un poids positif ou négatif à chaque mot du vocabulaire pour chaque étiquette possible, et qu'il calcule ensuite la somme des poids des mots contenus dans un texte, pondérés par la TF-IDF du mot donné.

On peut d'ailleurs observer ces poids directement dans le modèle :

In [ ]:
mot = "angry"
num_mot = vectorization.transform([mot]).argmax()
weights = model.layers[0].get_weights()[0][num_mot]
print(f"Poids pour le mot [{mot}]:")
for i, weight in enumerate(weights):
    print(f"{get_label(i):<10} {weight}")

# 2 couches et plus

Globalement, le modèle obtenu fonctionne plutôt bien. Cependant, le score de précision nous indique que près d'un tiers de ses prédictions sont erronnées. Nous aimerions l'améliorer, en lui permettant de faire des calculs plus complexes qu'une simple somme de poids. Pour cela, nous allons transformer notre perceptron en **perceptron multicouche** (MLP, *multilayer perceptron*). Il s'agit tout simplement de plusieurs perceptrons simples mis bout à bout :

<div><img src="img/mlp.png" width="500"/></div>

L'intérêt d'avoir plusieurs couches est que le réseau peut faire des opérations plus complexes : Si un simple perceptron à une couche lui permettait de faire une somme pondérée des scores des mots, ajouter une autre couche lui permet par exemple de calculer des sous-scores correspondant à certaines catégories de mots. D'ailleurs, un [théorème]([https://fr.wikipedia.org/wiki/Th%C3%A9or%C3%A8me_d%27approximation_universelle) dit qu'un réseau de neurones possédant deux couches suffisamment grandes (une couche cachée et une couche de sortie) est en théorie capable d'apprendre n'importe quelle fonction ! En pratique, les réseaux ont souvent plus de deux couches, mais qui sont moins grandes.

Savoir quelle fonction d'activation, combien de couches et quelle taille de couche utiliser est difficile; en pratique, même si quelques guides généraux existent, il s'agit surtout de tester différentes valeurs et de regarder ce qui marche le mieux...

On notera toutefois que pour les couches cachées, un principe général est d'utiliser une taille qui soit une puissance de 2 (2, 4, 8, 16, 32 etc. neurones).

Rajouter des couches se fait très simplement avec Keras, il suffit de définir leur fonction d'activation. Pour les couches intermédiaires d'un réseau de neurones, plusieurs fonctions d'activation existent, donc les plus courantes sont `relu`, `sigmoid` et `tanh`.

Voici donc un exemple à deux couches :

In [ ]:
keras.utils.set_random_seed(seed)
model2 = Sequential()

# Ajout d'une couche cachée avec 16 sorties, utilisant la fonction d'activation sigmoïde
model2.add(Dense(16, activation='sigmoid'))
# Ajout de la couche de sortie, avec 3 sorties, et utilisant le Softmax comme fonction d'application
model2.add(Dense(3, activation='softmax'))

# Entraînement et affichage
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model2.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
model2.summary()
display_accuracy(history)

**Q3 :** Faites quelques essais supplémentaires avec différentes tailles de couches, ainsi qu'en modifiant le nombre d'itérations. Vous pouvez également rajouter une couche supplémentaire similaire à la première, avec la taille de sortie de votre choix. Sans passer plus de 5-10 minutes sur cette question, arrivez-vous à obtenir une précision de validation qui soit supérieure à 70\% ?

Donnez un exemple de paramètres que vous avez testés et les résultats obtenus (meilleure précision de validation atteinte durant l'entraînement), puis calculez la précision atteinte sur le jeu de test grâce au code ci-dessous :

**Réponse :** Dépend de ce que vous avez. Il est possible mais très difficile de dépasser les 70% de précision.

In [ ]:
# Calcul de la précision sur le jeu de test
y_pred = model2(X_test.toarray()).numpy().argmax(axis=1)
y_true = y_test.argmax(axis=1)
print(f"Précision (test): {(y_pred == y_true).sum() / len(y_pred):.2%}")

**Attention :** On rappelle que bien que la précision globale soit une métrique utile, elle peut cacher des disparités entre les différentes classes. Selon les applications, il peut être préférable d'avoir un modèle qui soit globalement moins performant, mais qui soit aussi performant sur toutes les classes.

Observons donc la répartition des étiquettes dans le jeu de données :

In [ ]:
df['sentiment'].value_counts(normalize=True).plot(kind='bar')

Ici, les trois classes sont représentées avec des proportions assez égales dans le jeu de données. Cependant, il peut être utile d'avoir d'observer à quel point celle-ci peut varier au sein des classes :

In [ ]:
print(classification_report(y_true, y_pred, target_names=[get_label(x) for x in range(3)]))

**Q3 bis :** Comment varie la précision de votre modèle au sein des trois classes ?

**Réponse :** dépend de ce que vous avez. En général, la précision sur les exemples positifs et négatifs est assez bonne, et moins sur les exemples neutres; en effet, il est difficile de dire exactement ce qui rend un tweet neutre, alors que certains mots sont très liés à un sentiment positif ou négatif.

### Taille du réseau

À la fin de l'entraînement, Keras affiche le nombre de paramètres que contient votre modèle, c'est-à-dire son nombre de poids et biais. Le modèle de la semaine dernière contenait un total de 12 poids (matrice de taille (3, 4)) et 3 biais, soit 15 paramètres. Ici, notre modèle à une couche contient 79 194 paramètres à entraîner (*trainable params*), il est donc beaucoup plus gros ! C'est souvent le cas pour les modèles travaillant sur du texte, qui ont besoin de beaucoup de poids pour être capable de modéliser un vocabulaire entier.

De plus, le nombre de paramètres d'un modèle augmente avec sa profondeur (nombre de couches). Les modèles de langue tels que celui utilisé par ChatGPT contiennent plusieurs milliards, voire dizaines de milliards de paramètres. Plus un modèle a de paramètres, plus l'entraîner ou l'utiliser prend de la mémoire et du temps.

**Q3 ter :**  Combien de paramètres contient votre réseau de neurones ?

**Réponse :** dépend de ce ce que vous avez. Il faut bien lire la ligne *trainable params* (paramètres du réseau de neurones) et non pas la ligne *total params*, qui inclut les paramètres de l'optimiseur ne servant que pendant l'entraînement mais supprimés lors de l'inférence.

## Analyse des résultats

Le modèle semble ne pas avoir d'amélioration significative, malgré l'ajout d'une couche supplémentaire. Observons quelques-unes des erreurs faites par le modèle :

In [ ]:
max_errors = 10
errors = 0
for idx, row in df.iterrows():
    pred = predict(row['text'], model, False)
    true = row['sentiment']
    if pred != true:
        print(f'Prédiction : [{pred}] au lieu de [{true}] pour le tweet [{row["text"]}]')
        errors += 1
        if errors >= max_errors:
            break

**Q4 :** D'après vous, d'où peuvent venir les erreurs de classification faites par le modèle ? N'hésitez pas à traduire le contenu des tweets ci-dessus en français si besoin.

**Réponse :** On peut observer plusieurs phénomènes, dont entre autres :
- Mots détectés comme positifs utilisés dans un contexte négatif ou sarcastique ("fun")
- Composition de mots neutres donnant un sentiment positif ou négatif ("give in")
- Mots rares/argotiques rares dans le corpus d'entraînement
- Difficulté de savoir ce qu'est un tweet "neutre"

Il est possible de visualiser les résultats du modèle grâce à une **matrice de confusion**. Il s'agit d'un tableau contenant en abscisse les étiquettes prédites par le modèle, et en ordonnée les étiquettes véritables. Chaque case contient alors le nombre d'exemples correspondant à une prédiction donnée :

In [ ]:
y_pred = model(X_test.toarray()).numpy().argmax(axis=1)
y_true = y_test.argmax(axis=1)
ConfusionMatrixDisplay.from_predictions(y_true, y_pred, display_labels=[get_label(x) for x in range(3)])
plt.show()

Ici par exemple, la première rangée indique que 449 + 305 + 32 = 786 tweets étaient étiquetés comme négatifs dans le jeu de données. Sur ces 449, le modèle en a correctement étiqueté 449 comme négatifs, mais en a également étiqueté 305 comme neutres et 32 comme négatifs.

Similairement, la première colonne indique que sur 449 + 178 + 32 tweets que le modèle a décrit comme négatifs, seuls 449 étaient vraiment négatifs, 178 étaient en fait neutres, et 32 étaient positifs.

La jauge de couleur permet d'identifier facilement où sont les erreurs; dans l'idéal, la diagonale de la matrice de confusion doit être jaune, et le reste le plus violet possible. Ici, on remarque que le modèle a du mal à distinguer entre les tweets neutres et positifs/négatifs, mais qu'il étiquette très rarement un tweet négatif comme positif ou un positif comme négatif. D'autre part, la distinction entre un tweet négatif (ou positif) et un tweet neutre est assez floue et subjective. On peut donc imaginer que s'il n'y avait que ces deux types d'étiquettes, le modèle obtiendrait de meilleurs résultats.

# Bonus

Si vous avez terminé le TP avant la fin du temps imparti, voici deux tâches supplémentaires, que vous pouvez faire dans l'ordre de votre choix. Si vous les faites, gardez-en une trace dans votre notebook :

## Partie 1 : Critiques de films (en français)

Le fichier `data_b.csv` contient un autre jeu de données, cette fois-ci composé de critiques de films enregistrées sur AlloCiné (en français). Au texte de chaque critique est associé un score de sentiment qui vaut soit 0 (critique négative), soit 1 (critique positive).

Dans une nouvelle cellule (ou plusieurs), copiez le code de la partie principale de ce TP et adaptez-le afin d'entraîner un nouveau réseau de neurones sur les critiques de films. Très peu de choses sont à changer, la différence principale étant qu'il y a ici seulement deux étiquettes (positif/négatif) et non pas trois. En séparant toujours bien les données en jeux d'entraînement, de validation et de test, essayez ensuite d'entraîner un réseau de neurones afin d'obtenir les meilleures performances possibles. Il est facile d'obtenir une précision atteignant 90 à 91\%, pouvez-vous faire plus ? N'hésitez pas à expérimenter en changeant les couches, leur taille, leurs fonctions d'activation...

In [ ]:
# Lecture du fichier contenant les critiques de films... À vous d'écrire le reste !
df = pd.read_csv('data_b.csv')
df

## Partie 2 : Pré-traitement de texte avancé

La classe `TfidfVectorizer` a [beaucoup de paramètres optionnels](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). En particulier, elle permet de pré-traiter et nettoyer le texte avant d'obtenir sa TF-IDF, avec par exemple :
- La suppression des accents et caractères spéciaux (paramètre `strip_accents`)
- Le passage du texte en minuscules (paramètre `lowercase`)
- La suppression des mots courants tels que "the", "and", etc. (paramètre `stop_words`)
- L'enregistrement d'expression multimots : plutôt que de calculer la TF-IDF pour les mots individuels, on peut également la calculer pour des suites (n-grams) de 2 ou plus mots consécutifs (on peut par exemple imaginer que le mot "very" soit neutre, mais que les expressions "very good" ou "very bad" soient de plus forts indicateurs que seulement "good" et "bad"). Ceci peut être contrôlé grâce au paramètre `ngram_range`. Attention, si vous modifiez ce paramètre, il est fortement conseillé d'également utiliser le paramètre `max_features` pour limiter la taille du dictionnaire à (par exemple) 30 000 mots ou suites de mots, ou vous risquez de manquer de mémoire.
- Le fait d'ignorer des mots trop rares ou trop fréquents (paramètres `min_df` et `max_df`)

Tentez d'expérimenter avec ces paramètres afin de voir si vous arrivez à améliorer les performances du réseau de neurones (soit pour les tweets, soit pour les critiques de films).